<a href="https://colab.research.google.com/github/data-IA-2022/Airline_Tarik/blob/main/Airline_Tarik.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Sentiment Analysis - Airline Passenger Satisfaction


-----



<img src="https://simplonline.co/_next/image?url=https%3A%2F%2Fsimplonline-v3-prod.s3.eu-west-3.amazonaws.com%2Fmedia%2Fimage%2Fjpg%2F64814-shutterstock-1073953772-642497423efc3496249445.jpg&w=1280&q=75" width="500"/>

## Objectifs
A partir d'une enquête de satisfaction menée sur un ensemble de 130 000 passagers, vous devez mettre en oeuvre une démarche de machine learning : 
- pour comprendre quelles sont les informations qui influent sur la satisfaction / insatisfaction d'un passager
- pour prédire la satisfaction / insatisfaction d'un passager

## EDA (Exploratory Data Analysis) sur le jeu de données

In [3]:
# Mount drive
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


## 1.Le jeu de données
Le jeu de données est disponible ci-dessous.  
Il concerne la satisfaction client selon différentes caractéristiques :
* la cible à prédire est la colonne `Satisfaction`
* les features (numériques et catégorielles) sont toutes les autres colonnes :
  - `Age`:L'age des passagers
  -  `Gender`: Le genre des passagers (Femme, Homme)
  -  `Type of Travel`: Le but du voyage des passagers (Voyage personnel, Voyage d'affaires)
  -  `Class`: La classe de voyage dans l'avion des passagers (Affaires, Économique, Économique Plus)
  -  `Customer Type`: Le type de client (Client fidèle, client infidèle)
  -  `Flight distance`: La distance de vol de ce voyage
  -  `Inflight wifi service`: Le niveau de satisfaction du service wifi à bord (0: Non applicable; 1-5)
  -  `Ease of Online booking`: Le niveau de satisfaction de la réservation en ligne
  -  `Inflight service`: Le niveau de satisfaction du service à bord
  -  `Online boarding`: Le niveau de satisfaction de l'embarquement en ligne
  -  `Inflight entertainment`: Le niveau de satisfaction du divertissement à bord
  -  `Food and drink`: Le niveau de satisfaction de la nourriture et de la boisson
  -  `Seat comfort`: Le niveau de satisfaction du confort des sièges
  -  `On-board service`: Le niveau de satisfaction du service à bord
  -  `Leg room service`: Le niveau de satisfaction de l'espace pour les jambes
  -  `Departure/Arrival time convenient`: Le niveau de satisfaction de la convenance de l'heure de départ/arrivée
  -  `Baggage handling`: Le niveau de satisfaction de la gestion des bagages
  -  `Gate location`: Le niveau de satisfaction de l'emplacement de la porte
  -  `Cleanliness`: Le niveau de satisfaction de la propreté
  -  `Check-in service`: Le niveau de satisfaction du service d'enregistrement
  -  `Departure Delay in Minutes`: Les minutes de retard au départ
  -  `Arrival Delay in Minutes`: Les minutes de retard à l'arrivée
  -  `Flight cancelled`: Si le vol a été annulé ou non (Oui, Non)
  -  `Flight time in minutes`: Les minutes de la durée de vol

## 2.Les librairies


In [4]:
!pip install numpy==1.23
!pip install dython

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 31.4 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.22.4
    Uninstalling numpy-1.22.4:
      Successfully uninstalled numpy-1.22.4


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
# Import et traitement des données
import pandas as pd
import numpy as np

#correlation
from dython.nominal import associations
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm

# Graphiques
import seaborn as sns ; sns.set()
import matplotlib.pyplot as plt

# Machine learning - Preprocessing
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, RobustScaler, StandardScaler #MinMaxScaler, 
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.compose import ColumnTransformer

# Machine learning - Automatisation
from sklearn.pipeline import Pipeline
from sklearn import set_config

# Machine learning - Modèle d'apprentissage supervisé
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, HistGradientBoostingClassifier
from sklearn.decomposition import PCA

# Machine learning - Modèle selection
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold, cross_val_score, HalvingGridSearchCV

# Machine learning - Métriques d'erreur
from sklearn.metrics import confusion_matrix, accuracy_score, ConfusionMatrixDisplay, f1_score, fbeta_score

RuntimeError: ignored

ImportError: ignored

In [ ]:
# read csv
url = "https://raw.githubusercontent.com/remijul/dataset/master/Airline%20Passenger%20Satisfaction.csv"
df = pd.read_csv(url, encoding='utf-8', sep=';')
df.drop(['id'], axis=1, inplace=True)
df.columns = df.columns.str.replace(" ", "_")
df.columns = df.columns.str.replace("-", "_")
df.columns = df.columns.str.replace("/", "_")
df.head()


## 3.Analyse de données exploratoire (EDA)

### 3.1 Description du dataset

In [ ]:
# Dimmension du jeu de données
df.shape

In [ ]:
# Nom et types
df.dtypes


In [ ]:
# Infos
df.info()

###Données manquantes?


In [ ]:
df.isna().sum()

### 3.2 Analyse univariée 
Description statistique des données.


In [ ]:
df.describe(include='all')

### 3.3 Analyse bivariée - Variables numériques

Distribution de la target en fonction des variables numériques.

In [ ]:
#df_eda
df_eda = df.replace(['Male', 'Female'],[0, 1])
df_eda['Arrival_Delay_in_Minutes'] = df['Arrival_Delay_in_Minutes'].fillna(df['Departure_Delay_in_Minutes'])
# df_eda['Arrival Delay in Minutes'] = df_eda['Arrival Delay in Minutes'].astype('int64')
df_eda.info()

In [ ]:
# create a dataframe
dataframe = pd.DataFrame(df_eda, columns = ['Satisfaction',	'Gender',	'Customer', 'Type_Age','Departure_Delay_in_Minutes',	'Arrival_Delay_in_Minutes' ])

  
# selecting rows based on condition
rslt_df = dataframe[dataframe['Arrival_Delay_in_Minutes'].isna() ]
rslt_df

In [ ]:
fig, axs = plt.subplots(figsize=(16, 10))

# Plot variable 1
ax1 = plt.subplot(2,2,1)
sns.histplot(data=df_eda, x="Class", hue="Satisfaction", kde=True)

# Plot variable 2
ax1 = plt.subplot(2,2,2)
sns.histplot(data=df_eda, x="Age", hue="Satisfaction", kde=True)

# Plot variable 3
ax1 = plt.subplot(2,2,3)
sns.histplot(data=df_eda, x="Flight_Distance", hue="Satisfaction", kde=True)

# Plot variable 4
ax1 = plt.subplot(2,2,4)
sns.histplot(data=df_eda, x="Type_of_Travel", hue="Satisfaction", kde=True)




### 3.4 Data correlation

In [ ]:
# Identification des colonnes categorielle avec dython comme le select_dtypes(include=['object']) de pandas
from dython.nominal import identify_nominal_columns
categorical_features=identify_nominal_columns(df_eda)
categorical_features

In [ ]:
#option de la fonction association qui retourne la matrice de correlation corr et le graphique de cette dernière ax (matplotlib)
#   associations(dataset, nominal_columns='auto', numerical_columns=None, mark_columns=False, nom_nom_assoc='cramer', num_num_assoc='pearson', bias_correction=True, nan_strategy=_REPLACE, nan_replace_value=_DEFAULT_REPLACE_VALUE, ax=None, figsize=None, annot=True, fmt='.2f', cmap=None, sv_color='silver', cbar=True, vmax=1.0, vmin=None, plot=True, compute_only=False, clustering=False, title=None, filename=None)

In [ ]:
complete_correlation= associations(df_eda,  filename= 'complete_correlation.png', figsize=(16,16))

In [ ]:
# convert categorical variables to numerical variables
# df_eda = pd.get_dummies(df_eda)

# create correlation matrix
# corr_matrix = df_eda.corr()
corr_matrix = complete_correlation['corr']
# # select only columns with correlation > 0.3 with the target variable 'Satisfaction'
# corr_cols = corr_matrix.loc[corr_matrix['Satisfaction_satisfied'] > 0.25, 'Satisfaction_satisfied'].sort_values(ascending=False)
corr_cols = corr_matrix.loc[corr_matrix['Satisfaction'] > 0.22, 'Satisfaction'].sort_values(ascending=False)
# print the resulting correlation matrix
corr_cols

##Anova

In [ ]:
from scipy.stats import f_oneway

def anova_test(df, target_col):
    """
    Performs ANOVA test between each numerical feature in the dataframe and the target variable
    specified by target_col and returns a new dataframe containing the F-score and p-value
    for each test.
    
    Args:
    df (pandas.DataFrame): the input dataframe
    target_col (str): the name of the target column in the dataframe
    
    Returns:
    pandas.DataFrame: a dataframe containing the F-score and p-value for each ANOVA test.
    """
    p_values = []
    f_scores = []
    columns = []
    
    # Perform ANOVA test for each numerical column in the dataframe
    for col in df.select_dtypes(include=['float64', 'int64']).columns:
        if col != target_col:
            category_groups = [df[df[target_col] == category][col] for category in df[target_col].unique()]
            f, p = f_oneway(*category_groups)
            p_values.append(p)
            f_scores.append(f)
            columns.append(col)
    
    # Create a new dataframe with the results
    results_df = pd.DataFrame({'Feature': columns, 'F-score': f_scores, 'p-value': p_values})
    return results_df

In [ ]:
df.head(1)

In [ ]:
anova_df = anova_test(df_eda, "Satisfaction")
anova_df

In [ ]:
sorted_features = anova_df.sort_values(by=['F-score'], ascending=False)['Feature'].tolist()
sorted_features

###On observe que:
les features les plus corrélées avec la satisfaction client sont les services à bord avec:
 - divertissement à bord,
 - la class,
 - l'espace pour les jambes(normale ^^)
 - baggages,
 - enregistrement,
 - propreté

et les services en lignes:
 - le support en ligne,
 - Online_boarding


In [ ]:
#matrice correlation
sns.heatmap(df_eda[['Inflight_entertainment', 'Online_support', 'Ease_of_Online_booking',
                    'On_board_service', 'Leg_room_service', 'Baggage_handling', 'Checkin_service',
                    'Cleanliness', 'Online_boarding']].corr(), annot=True, linewidths=0.5)

Les features 

## 4.Preprocessing

### 4.1 Preprocessing sur la cible

Sélection de la variable cible (target) `Satisfaction`.

In [ ]:
y = df['Satisfaction']

### 4.2 Preprocessing sur les features

Features preprocessing

In [ ]:
X = df.drop(columns='Satisfaction')
X.head()

#### 4.2.1 Variables catégorielles

Selection des variables categorielles.

In [ ]:
column_cat = X.select_dtypes(include=['object']).columns
column_cat

Déclaration de la méthode de preprocessing sur les variables catégorielles.

In [ ]:
transfo_cat = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output = False))
])

#### 4.2.2 Variables numériques

Selection des variables numériques.

In [ ]:
column_num = X.select_dtypes(exclude=['object']).columns
column_num

Déclaration de la méthode de preprocessing sur les variables numériques.

In [ ]:
transfo_num = Pipeline(steps=[
    ('imputation', KNNImputer(n_neighbors=3, weights="uniform")),
    ('scaling', StandardScaler())
])

#### 4.2.3 Transformation des features

In [ ]:
preparation = ColumnTransformer(
    transformers=[
        ('data_cat', transfo_cat , column_cat),
        ('data_num', transfo_num , column_num)
    ])

In [ ]:
set_config(display="diagram")
preparation

### 4.3 Répartition `train-test-split` 

In [ ]:
# Noter la présence du stratify sur la target pour mieux équilibrer les observation sur la target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2, stratify=y)

## 5.Entrainement du modèle


### 5.1 Premier essai : `KNeighboursClassifier`

In [ ]:
clf = KNeighborsClassifier()

#### 5.1.1 Déclaration du modèle
Déclaration du modèle sans paramétrage.
Documentation [KNC()](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html#sklearn.neighbors.KNeighborsClassifier).

#### 5.1.2 Intégration dans le pipeline
Intégration du modèle d'apprentissage dans le pipeline.

In [ ]:
model = Pipeline(steps=[('preparation', preparation),
                        ('model', clf)])
model

#### 5.1.3 Apprentissage
Phase d'apprentissage du modèle.

In [ ]:

cv = StratifiedKFold(n_splits=5)
# Fit the pipeline to the training data and evaluate it using cross-validation
model.fit(X_train, y_train)
scores = cross_val_score(model, X_train, y_train, cv=cv)

# Print the mean score and the standard deviation
print("Accuracy: %0.3f (+/- %0.3f)" % (scores.mean(), scores.std() * 2))

#### 5.1.4 Prédictions
Utiliser du jeu de test pour réaliser les prédictions.

In [ ]:
y_pred = model.predict(X_test)
y_pred

#### 5.1.5 Evaluation
Evaluation de la performance (`accuracy score`) du modèle.


In [ ]:
score = accuracy_score(y_test, y_pred)
print(f"Performance du modèle {clf} - Accuracy score :", round(score, 5))

Matrice de confusion

In [ ]:
conf_matrix = confusion_matrix(y_true=y_test, y_pred=y_pred)
conf_matrix

In [ ]:
sns.set_style("dark")

cm_plot = ConfusionMatrixDisplay(conf_matrix,
                                display_labels=df['Satisfaction'].unique())

cm_plot.plot()

## 6.Comparaison de modèles

### 6.1 Construction d'une fonction `getClassifResults()`
Pour faciliter le processus complet d'entrainement et d'évaluation du modèle, nous allons créer ici une fonction qui fait l'intégralité du processus d'apprentissage supervisé et qui restitue les performances du modèle.  

La fonction `getClassifResults()` prend comme paramètres d'entrée :  
* `classifier` : le modèle de classification que nous souhaitons utiliser.
* `parameters` : la liste des hyper-paramètres et de leurs valeurs qui souhaitons étudier dans le processus `GridSearchCV`, au format dictionnaire.
* `data` : la dataframe sur laquelle nous souhaitons réaliser le processus d'apprentissage supervisé.

In [ ]:
def getClassifResults(classifier, parameters, data):

  # PREPROCESSING
  # Target
  y = data['Satisfaction']
  
  # Features preprocessing
  X = data.drop(columns='Satisfaction')
  
  column_cat = X.select_dtypes(include=['object']).columns
  transfo_cat = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse = False))
  ])
  
  column_num = X.select_dtypes(exclude=['object']).columns
  transfo_num = Pipeline(steps=[
    ('imputation', KNNImputer(n_neighbors=3, weights="uniform")),
    ('scaling', StandardScaler())
  ])

  preparation = ColumnTransformer(
    transformers=[
        ('data_cat', transfo_cat , column_cat),
        ('data_num', transfo_num , column_num)
    ])

  # train-test-split	
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2, stratify=y)	

  # Pipeline and Model
  model = Pipeline(steps=[('preparation', preparation),
                          ('model', classifier)])

  # Define the cross-validation strategy
  cv = StratifiedKFold(n_splits=5)

  # Gridsearch
  grid = GridSearchCV(estimator = model, param_grid = parameters, scoring = 'f1_weighted', cv = cv, n_jobs =-1, verbose = 2)
  
  #grid = GridSearchCV(estimator = model, param_grid = parameters, scoring = 'f1_weighted', cv = 5, n_jobs =-1, verbose = 0)

  # Fit
  grid.fit(X_train, y_train)

  # Predict
  y_pred = grid.predict(X_test)
  y_train_pred = grid.predict(X_train)
  train_accu_score = accuracy_score(y_train, y_train_pred)
  test_f1_score = fbeta_score(y_test, y_pred, average='weighted', beta=0.5)
  test_accu_score = accuracy_score(y_test, y_pred)
  # Results
  classifier_results = []
  classifier_results.append(grid.cv_results_['mean_fit_time'].mean().round(4))
  classifier_results.append(grid.best_score_.round(4))
  classifier_results.append(train_accu_score.round(4))
  classifier_results.append(test_f1_score.round(4))
  classifier_results.append(test_accu_score.round(4))
  classifier_results.append(grid.best_params_)

  return  classifier_results

##création d'un dictionnaire de model a tester

In [ ]:
classifiers = {
    'KNeighborsClassifier': {'model__n_neighbors': range(1, 10, 2)},
    'RandomForestClassifier': {'model__n_estimators': range(50, 500, 50), 'model__criterion': ('gini', 'entropy')},
    'HistGradientBoostingClassifier': {'model__learning_rate': np.arange(start=0.1, stop=0.9, step=0.1), 'model__loss': ('auto', 'binary_crossentropy')}
}

##Itération dans le dict et utilisation de la fonction, save dans un dict

In [ ]:
results = {}
for classifier_name in classifiers.keys():
    classifier = globals()[classifier_name]()
    parameters = classifiers[classifier_name]
    res = getClassifResults(classifier, parameters, data=df)
    results[classifier_name] = res

##Génération du rapport avec plot matrice de confusion

In [ ]:
results_df = pd.DataFrame(index=['Testing time (sec)', 'Training score (F1-score)', 'Training score (accuracy stratKfold)', 'Evaluation score (F1-score)', 'Evaluation score (accuracy stratKfold)', 'Best parameters'])
confusion_matrices = {}

for classifier_name in results.keys():
    res = results[classifier_name]
    results_df[classifier_name] = res
    classifier = globals()[classifier_name]()
    classifier.fit(X_train, y_train)
    y_pred = classifier.predict(X_test)
    conf_matrix = confusion_matrix(y_true=y_test, y_pred=y_pred)
    confusion_matrices[classifier_name] = conf_matrix

for classifier_name in confusion_matrices.keys():
    conf_matrix = confusion_matrices[classifier_name]
    cm_plot = ConfusionMatrixDisplay(conf_matrix, display_labels=df['Satisfaction'].unique())
    cm_plot.plot()

### 6.4 KNeighborsClassifier
Nous allons utiliser ici le KNeighborsClassifier associé à sa grille de paramètres spécfiques.

In [ ]:
classifier = KNeighborsClassifier()
parameters = {
    'model__n_neighbors' : range(1, 10, 2)
}

res_KNN = getClassifResults(classifier, parameters, data=df)
print(res_KNN)

### 6.5 Random Forest Classifier


In [ ]:
classifier = RandomForestClassifier()
parameters = {
    'model__n_estimators' : range(50, 500, 50),
    'model__criterion' : ('gini', 'entropy')
}

res_RF = getClassifResults(classifier, parameters, data=df)
print(res_RF)

### 6.6 HistGradient Boosting Classifier
Nous allons utiliser ici le Gradient Boosting Classifier associé à sa grille de paramètres spécfiques.

In [ ]:
classifier = HistGradientBoostingClassifier()
parameters = {
    'model__learning_rate' : np.arange (start=0.1, stop=0.9, step=0.1),
    'model__loss' : ('auto', 'binary_crossentropy')
    }

res_HGB = getClassifResults(classifier, parameters, data=df)
print(res_HGB)

### 6.7 Synthèse
Nous allons conserver et afficher tous les résulats dans une dataframe.

In [ ]:
results_df = pd.DataFrame(res_KNN,
                          index=['Testing time (sec)', 'Training score (F1-score)', 'Training score (accuracy stratKfold)', 'Evaluation score (F1-score)', 'Evaluation score (accuracy stratKfold)', 'Best parameters'],
                          columns=['KNeighborsClassifier'])
results_df['HistGradient Boosting Classif'] = res_HGB
results_df['Random Forest Classif'] = res_RF

In [ ]:
results_df